In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np

In [2]:
data = np.concatenate([np.genfromtxt('../data/dis_forautoencoder_2012_dec_tokyo/day_{}.csv'.format(i), delimiter=',', dtype=np.int32)[:, 1:] for i in range(1, 32)], axis=0)
np.random.shuffle(data)

data_size = data.shape[0]
train_data = data[:int(data_size * 0.8)]
valid_data = data[int(data_size * 0.8):]

In [21]:
class RNNPredictor(nn.Module):
    def __init__(self, num_loc, embedding_dim, hidden_dim, n_layers):
        
        super(RNNPredictor, self).__init__()
        
        self.embedding = nn.Embedding(num_loc, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=n_layers, batch_first=True)
        self.out = nn.Linear(hidden_dim, num_loc - 2)
    
    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.gru(embedded, None)
        return self.out(output)

In [28]:
NUM_LOC = 1443
EMBEDDING_DIM = 64
HIDDEN_DIM = 256
N_LAYERS = 2

rnn_predictor = RNNPredictor(NUM_LOC, EMBEDDING_DIM, HIDDEN_DIM, N_LAYERS).cuda()
optimizer = torch.optim.SGD(rnn_predictor.parameters(), lr=1e-2, momentum=0.9)
optim_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [2, 4, 6, 8], gamma=0.1)

In [29]:
criteria = nn.CrossEntropyLoss(ignore_index=-1)

In [ ]:
BATCH_SIZE = 64
for it in range(1, 11):
    optim_scheduler.step()
    np.random.shuffle(train_data)
    avg_loss = 0.0
    valid_loss = 0.0
    cnt = 0
    for i in range(0, train_data.shape[0], BATCH_SIZE):
        cnt += 1
        perc = i / train_data.shape[0] * 100.0
        x_batch = train_data[i: min(train_data.shape[0], i + BATCH_SIZE)]
        y_batch = Variable(torch.LongTensor(x_batch[:, 1:])).cuda()
        x_batch = Variable(torch.LongTensor(x_batch[:, :-1])).cuda()
        y_pred = rnn_predictor(x_batch)
        optimizer.zero_grad()
        loss = criteria(y_pred.view(-1, NUM_LOC - 2), torch.clamp(y_batch - 2, min=-1).view(-1))
        loss.backward()
        avg_loss += loss.data[0]
        optimizer.step()
        print('Iteration {:03d}, {:3.1f}%, avg_loss = {:.4f}'.format(it, perc, avg_loss / cnt), end='\r')
    print('')
    
    cnt = 0
    for i in range(0, valid_data.shape[0], BATCH_SIZE):
        cnt += 1
        x_batch = valid_data[i: min(valid_data.shape[0], i + BATCH_SIZE)]
        y_batch = Variable(torch.LongTensor(x_batch[:, 1:])).cuda()
        x_batch = Variable(torch.LongTensor(x_batch[:, :-1])).cuda()
        y_pred = rnn_predictor(x_batch)
        loss = criteria(y_pred.view(-1, NUM_LOC - 2), torch.clamp(y_batch - 2, min=-1).view(-1))
        valid_loss += loss.data[0]
    print('Iteration {:03d}, avg_loss = {:.4f}'.format(it, valid_loss / cnt))
    torch.save(rnn_predictor, '../results/sadRNNPredictor/rnn_predictor')

Iteration 001, 100.0%, avg_loss = 2.0857
Iteration 001, avg_loss = 2.0276


/home/fan/anaconda3/envs/py3.6/lib/python3.6/site-packages/torch/serialization.py:158: UserWarning: Couldn't retrieve source code for container of type RNNPredictor. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Iteration 002, 100.0%, avg_loss = 2.0161
Iteration 002, avg_loss = 2.0141
Iteration 003, 100.0%, avg_loss = 2.0078
Iteration 003, avg_loss = 2.0065
Iteration 004, 100.0%, avg_loss = 2.0028
Iteration 004, avg_loss = 2.0053
Iteration 005, 100.0%, avg_loss = 2.0020
Iteration 005, avg_loss = 2.0045
Iteration 006, 100.0%, avg_loss = 2.0016
Iteration 006, avg_loss = 2.0044
Iteration 007, 100.0%, avg_loss = 2.0014
Iteration 007, avg_loss = 2.0043
